In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime

# Using some Twitter data
It can be downloaded here: https://www.isi.edu/~lerman/downloads/twitter/twitter2010.html

In [2]:
# some lines in the data set are broken, let's ignore them
df = pd.read_csv("./data/link_status_search_with_ordering_real.csv", sep=',', error_bad_lines=False, nrows=500000)
df['datetime'] = pd.to_datetime(df.create_at_long, unit='ms')

Skipping line 153859: expected 11 fields, saw 12

Skipping line 431580: expected 11 fields, saw 12
Skipping line 431962: expected 11 fields, saw 12

Skipping line 498662: expected 11 fields, saw 12



In [3]:
df.head()

,link,id,create_at,create_at_long,inreplyto_screen_name,inreplyto_user_id,source,bad_user_id,user_screen_name,order_of_users,user_id,datetime
0,http://gd.is/4nfm,25435023138,2010-09-24 00:00:00,1285358980000,\N,-1,"<a href=""http://mobile.twitter.com"" rel=""nofol...",119586070,barra_fake,616,147240385,2010-09-24 20:09:40
1,http://gd.is/4nfm,25439610259,2010-09-24 00:00:00,1285363164000,\N,-1,"<a href=""http://mobile.twitter.com"" rel=""nofol...",119586070,barra_fake,652,147240385,2010-09-24 21:19:24
2,http://is.gd/f4wDA,25444403307,2010-09-24 00:00:00,1285367458000,\N,-1,"<a href=""http://tinyurl.com/3xakwys"" rel=""nofo...",119586070,barra_fake,3009,147240385,2010-09-24 22:30:58
3,http://is.gd/f4wDA,25451679785,2010-09-24 00:00:00,1285373674000,\N,-1,"<a href=""http://tinyurl.com/3xakwys"" rel=""nofo...",119586070,barra_fake,3182,147240385,2010-09-25 00:14:34
4,http://lc4.in/2KAJ,25446362980,2010-09-24 00:00:00,1285369191000,\N,-1,"<a href=""http://www.apple.com"" rel=""nofollow"">...",119586070,barra_fake,4580,147240385,2010-09-24 22:59:51


In [4]:
def summarize_user(user_data):
    norm_dates = user_data.datetime.dt.normalize()
    ndays_length = (norm_dates.max() - norm_dates.min()).days + 1
    ndays_act = norm_dates.nunique()
    return pd.Series({'n_tweets': len(user_data),
                      'n_days':ndays_act,
                      'n_links': user_data.link.nunique()
                     })

In [5]:
print datetime.now()
summary = df.groupby('user_id').apply(summarize_user)
print datetime.now()

2018-05-06 12:30:55.824737
2018-05-06 12:31:36.534451


In [6]:
summary

,n_days,n_links,n_tweets
user_id,,,
917,1,1,2
951,1,1,1
978,1,1,1
1047,1,1,1
1186,1,1,1
1317,1,1,1
1501,2,2,2
1503,1,1,1
1546,1,1,1


In [7]:
import ipyparallel as ip

In [8]:
client = ip.Client()
dview = client.direct_view()

/usr/local/lib/python2.7/site-packages/ipyparallel/util.py:208: RuntimeWarning: IPython could not determine IPs for tux.local: [Errno 8] nodename nor servname provided, or not known
  RuntimeWarning)
/usr/local/lib/python2.7/site-packages/ipyparallel/client/client.py:458: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@tux.local')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [9]:
users = np.array_split(df.user_id.unique(), 4)
dfs = [df[df.user_id.isin(_)] for _ in users]
dview.scatter('data', dfs)

<AsyncResult: scatter>

In [10]:
%%px --block
import sys
import numpy as np
import pandas as pd

In [11]:
%%px
def summarize_user(user_data):
    norm_dates = user_data.datetime.dt.normalize()
    ndays_length = (norm_dates.max() - norm_dates.min()).days + 1
    ndays_act = norm_dates.nunique()
    return pd.Series({'n_tweets': len(user_data),
                      'n_days':ndays_act,
                      'n_links': user_data.link.nunique()
                     })

In [12]:
print datetime.now()

2018-05-06 12:31:42.585878


In [13]:
%%px
df = data[0]
user_summary = df.groupby('user_id').apply(summarize_user)

In [14]:
%pxresult

In [15]:
summary  = pd.concat(dview.gather('user_summary').get())

In [16]:
print datetime.now()

2018-05-06 12:32:02.252318


In [17]:
summary

,n_days,n_links,n_tweets
user_id,,,
1186,1,1,1
285903,1,1,1
769594,1,1,1
774292,1,2,2
818377,1,1,1
945451,1,1,1
1046661,1,2,2
1649981,1,1,1
2015141,1,1,1
